In [ ]:
%matplotlib inline
from functools import partial
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import clemb
from clemb.forward_model import Forwardmodel
from clemb.syn_model import SynModel
from clemb.uks import UnscentedKalmanSmoother

from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints, unscented_transform
from scipy.optimize import brentq

import param
import panel as pn

import IPython.core.debugger
dbg = IPython.core.debugger.Pdb()
pn.extension()

In [ ]:
from bqplot.scales import LinearScale, DateScale, LogScale
from bqplot.interacts import PanZoom
from bqplot.marks import *
from bqplot.axes import Axis
from bqplot.figure import Figure
from bqplot.toolbar import Toolbar

from ipywidgets import VBox, HBox
import ipywidgets as widgets

import xarray as xr

import numpy as np
import pandas as pd
from clemb.syn_model import SynModel

In [ ]:
xds = SynModel(seed=42).run(1000., nsteps=100, integration_method='rk4', 
                    gradient=True, mode='gamma', addnoise=True,
                    estimatenoise=False)

In [ ]:
def get_min_max(lines, pad=.5, rangefactor=3):
    if False:
        lrange = max(.1, np.nanmax(lines[2]) - np.nanmin(lines[1]))
        # clip range at rangefactor*maximum range
        max_range = rangefactor * max(.1, np.nanmax(lines[0]) - np.nanmin(lines[0]))
        lrange = min(lrange, max_range)
        ymin = max(0., np.nanmin(lines[0]) - pad*lrange)
        ymax = np.nanmax(lines[0]) + pad*lrange
    else:
        lrange = max(.1, np.nanmax(lines[0]) - np.nanmin(lines[0]))
        ymin = max(0., np.nanmin(lines[0]) - pad*lrange)
        ymax = np.nanmax(lines[0]) + pad*lrange
    return (ymin, ymax)

def plot_ukf(data, color1='#1f77b4', color2='#ff7f0e'):
    f_options = []
    for o in list(data.data_vars):
        if not o.endswith('_err'):
            f_options.append(o)
    
    feature_x = widgets.Dropdown(description='Feature 1:')
    feature_y = widgets.Dropdown(description='Feature 2:')
    feature_x.options = f_options
    feature_y.options = f_options
    feature1 = feature_x.options[0]
    feature2 = feature_y.options[-1]
    feature_y.value = feature2

    traces = {}
    for o in f_options:
        if o+'_err' in list(data.data_vars):
            # ignore warnings due to NaNs
            with np.errstate(invalid='ignore'):
                ymean = data[o].values
                ymean = np.where(ymean < 0., 0., ymean)
                yerr = data[o+'_err'].values
                ymin = ymean - 3*yerr
                ymin = np.where(ymin < 0., 0., ymin)
                ymax = ymean + 3*yerr
                traces[o] = [ymean, ymin, ymax]
        else:
            ymean = data[o].values
            with np.errstate(invalid='ignore'):
                ymean = np.where(ymean < 0., 0., ymean)
            traces[o] = [ymean]
            
    xs = DateScale()
    try:
        dates = data['dates'].values
    except KeyError:
        dates = data['index'].values
    y1 = traces[feature1]
    y2 = traces[feature2]
    y1min, y1max = get_min_max(y1)
    y2min, y2max = get_min_max(y2)

    ys = LinearScale(min=y1min, max=y1max)
    panzoom = PanZoom(scales={'x': [xs], 'y': [ys]})
    x_axis = Axis(scale=xs, label='Date')
    y_axis_left = Axis(scale=ys, orientation='vertical', label=feature1,
                       label_color=color1)
    line_1 = Lines(x=dates, y=y1, scales={'x':xs, 'y':ys}, colors=[color1], fill='between', fill_opacities=[0, .5, .5])

    ys1 = LinearScale(min=y2min, max=y2max)
    y_axis_right = Axis(scale=ys1, orientation='vertical', label=feature2, side='right', grid_lines='none',
                        label_color=color2)
    line_2 = Lines(x=dates, y=y2, scales={'x':xs, 'y':ys1}, colors=[color2], fill='between', 
                   fill_opacities=[0, .5, .5], line_style='dashed')


    fig = Figure(marks=[line_1, line_2], axes=[x_axis, y_axis_left, y_axis_right])
    tb = Toolbar(figure=fig)

    def update_plot(a):
        y1 = traces[feature_x.value]
        y2 = traces[feature_y.value]
        y1min, y1max = get_min_max(y1)
        y2min, y2max = get_min_max(y2)
        ys.min = y1min
        ys.max = y1max
        ys1.min = y2min
        ys1.max = y2max
        basis1 = feature_x.value.split('_')[0]
        basis2 = feature_y.value.split('_')[0]
        if basis1 == basis2:
            ys1.min = ys.min
            ys1.max = ys.max
        line_1.y = y1
        line_2.y = y2
        y_axis_left.label = feature_x.value
        y_axis_right.label = feature_y.value

    feature_x.observe(update_plot, 'value')
    feature_y.observe(update_plot, 'value')

    return VBox([HBox([feature_x, feature_y]), VBox([fig, tb])])

plot_ukf(xds)

## Synthetic data

In [ ]:
uks = UnscentedKalmanSmoother(data=xds.to_dataframe())
Q = np.eye(9)*[1e-3, 1e-3, 1e-3, 1e-10, 1e1, 1e1, 1e3, 1e4, 1e4]*uks.dt*uks.dt
(T, M, X, Mi, Mo, qi) = uks.data.iloc[0][['T', 'M', 'X', 'Mi', 'Mo', 'qi']]
dqi = 1e-1
dMi = 1e-1
dMo = 1e-1
P0 = np.eye(9)*1e3
X0 = [T, M, X, qi*0.0864, Mi, Mo, dqi, dMi, dMo]
log_lh = uks(Q, X0, P0, test=True)
print(log_lh)

In [ ]:
plot_ukf(uks.data.to_xarray())

In [ ]:
if False:
    from nsampling import NestedSampling, Uniform, Normal

    T_Q = Uniform('T_Q', 1e-5, 1e-1)
    M_Q = Uniform('M_Q', 1e-5, 1e-1)
    X_Q = Uniform('X_Q', 1e-5, 1e-1)
    qi_Q = Uniform('qi_Q', 1e-8, 1.)
    Mi_Q = Uniform('Mi_Q', 1e-8, 1.)
    Mo_Q = Uniform('Mo_Q', 1e-8, 1.)
    dqi_Q = Uniform('dqi_Q',1e-1, 1e6)

    ns = NestedSampling()
    rs = ns.explore([T_Q, M_Q, X_Q, qi_Q, Mi_Q, Mo_Q, dqi_Q], 10, 300, likelihood=_fn)


In [ ]:
if False:
    [T_Q, M_Q, X_Q, qi_Q, Mi_Q, Mo_Q, dqi_Q] = rs.get_samples()[-1].get_value()
    [T_Q, M_Q, X_Q, qi_Q, Mi_Q, Mo_Q, dqi_Q] = rs.getexpt()
    dt = (df.index[1] - df.index[0])/pd.Timedelta('1D')
    Q = np.eye(7)*[T_Q, M_Q, X_Q, qi_Q, Mi_Q, Mo_Q, dqi_Q]*dt*dt
    xs, ps, Xs, Ps, log_lh = smooth(df, Q)
    log_lh

In [ ]:
if False:
    maxlog = 1e-30
    lhs = []
    zs = []
    for s in rs.get_samples():
        lh = s.get_logL()
        lhs.append(lh)
        zs.append(s.get_logZ())
        if lh>maxlog:
            maxlog = lh
            vals = s.get_value()

    [T_Q, M_Q, X_Q, qi_Q, Mi_Q, Mo_Q, dqi_Q] = vals
    Q = np.eye(7)*[T_Q, M_Q, X_Q, qi_Q, Mi_Q, Mo_Q, dqi_Q]*dt*dt
    xs, ps, Xs, Ps, log_lh = smooth(df, Q)
    plt.plot(zs)
    plt.plot(lhs)

## Real data

In [ ]:
from clemb.data import LakeData

In [ ]:
ld = LakeData()
df_rd = ld.get_data_fits('2018-11-01', '2019-09-11', smoothing='dv')

In [ ]:
T0, M0, C0 = df_rd.iloc[0][['T', 'M', 'X']].values
Mi0 = 10.
Mo0 = 10.
qi0 = 200.
dqi0 = 1e-1
dMi0 = 1e-1
dMo0 = 1e-1
dt = (df_rd.index[1] - df_rd.index[0])/pd.Timedelta('1D')
Q = np.eye(9)*[1e-2, 1e2, 1e-3, 1e-10, 1, 1, 1, 1e2, 1]*dt*dt
X0 = [T0, M0, C0, qi0*0.0864, Mi0, Mo0, dqi0, dMi0, dMo0]
P0 = np.eye(9)*[0.1*T0, 0.1*M0, 0.5*C0, 100*0.0864, 1e2, 1e2, 1., 1., 1.]
uks1 = UnscentedKalmanSmoother(data=df_rd)
log_lh = uks1(Q, X0, P0)
print(log_lh)

In [ ]:
import xarray as xr
rs = xr.open_dataset('../../../tests/data/forward_2019-01-01_2019-08-26.nc')
rs = rs.reindex(dict(dates=df_rd.index))
q_ns = rs['exp'].loc[:,'q_in']
q_ns_err = rs['var'].loc[:, 'q_in']

In [ ]:
uks1.data['qi_ns'] = q_ns.data
uks1.data['qi_ns_err'] = np.sqrt(q_ns_err.data)

In [ ]:
plot_ukf(uks1.data.to_xarray())

In [ ]:
if False:
    from nsampling import NestedSampling, Uniform, Normal
    
    def likelihood(var, sid, data):
        dt = (data.index[1] - data.index[0])/pd.Timedelta('1D')
        T_Q = var[0]
        M_Q = var[1]
        X_Q = var[2]
        qi_Q = var[3]
        Mi_Q = var[4]
        Mo_Q = var[5]
        dqi_Q = var[6]
        dMi_Q = var[7]
        dMo_Q = var[8]
        Q = np.eye(9)*[T_Q, M_Q, X_Q, qi_Q, 
                       Mi_Q, Mo_Q, dqi_Q,
                       dMi_Q, dMo_Q]*dt*dt
        T0, M0, X0 = data.iloc[0][['T', 'M', 'X']]
        Mi0 = 10.
        Mo0 = 10.
        dqi = 1e-1
        qi0 = 100.
        dqi0 = 1e-1
        dMi0 = 1e-1
        dMo0 = 1e-1
        X0 = [T0, M0, X0, qi0*0.0864, Mi0, Mo0, dqi0, dMi0, dMo0]
        try:
            xs, ps, Xs, Ps, log_lh = uks(data, Q, X0)
        except:
            return 1e-10
        return log_lh

    _lh = partial(likelihood, data=df_rd)
   

    T_Q = Uniform('T_Q', 1e-4, 1e-1)
    M_Q = Uniform('M_Q', 1e-4, 1e-1)
    X_Q = Uniform('X_Q', 1e-4, 1e-1)
    qi_Q = Uniform('qi_Q', 1e-8, 1.)
    Mi_Q = Uniform('Mi_Q', 1e-3, 1e2)
    Mo_Q = Uniform('Mo_Q', 1e-3, 1e2)
    dqi_Q = Uniform('dqi_Q',1., 1e2)
    dMi_Q = Uniform('dMi_Q', 1., 1e2)
    dMo_Q = Uniform('dMo_Q', 1., 1e2)
    ns_vars = [T_Q, M_Q, X_Q, qi_Q, Mi_Q,
               Mo_Q, dqi_Q, dMi_Q, dMo_Q]
    ns = NestedSampling()
    rs_ns = ns.explore(ns_vars, 10, 300, likelihood=_lh)